# 第四章 在 LangChain 中调用 OpenAI Function Calling

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、Pydantic语法](#二、Pydantic语法)
     - [2.1 简单创建Python类](#2.1-简单创建Python类)
     - [2.2 使用Pydantic创建类](#2.2-使用Pydantic创建类)
 - [三、 基于Pydantic的OpenAI函数定义](#三、基于Pydantic的OpenAI函数定义)
     - [3.1 构造OpenAI的function](#3.1-构造OpenAI的function)
     - [3.2 通过langchain使用function](#3.2-通过langchain使用function)
     - [3.3 强制使用function](#3.3-强制使用function)
 - [四、使用function](#四、使用function)
     - [4.1 链式使用](#3.1-链式使用)
     - [4.2 使用多个function](#4.2-使用多个function)

为了大家实验方便，关于以下示例，我们将使用硅基流动的API，于是相关的代码也会修改。

# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

# 二、Pydantic语法

Pydantic 是 python 的“数据验证库”。
- 与 python 类型注释一起工作。但是，与静态类型检查不同，它们在运行时被积极地用于数据验证和转换。
- 提供内置方法来序列化/反序列化模型到 JSON ，字典等。
- LangChain 利用 Pydantic 创建 JsONScheme 描述函数。

## 2.1 简单创建Python类

在标准python中，可以创建一个`User`类，拥有`name`、`age`、`email`三种属性值。直接进行创建，不能对字段类型进行约束，即年龄中可能传入不合规的字符串类型。

In [1]:
from typing import List
from pydantic import BaseModel, Field

In [2]:
# 创建User类
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [3]:
# 生成user对象
foo = User(name="Joe",age=32, email="joe@gmail.com")

# 输出foo中 name字段
print(foo.name)

Joe


In [4]:
# 生成user对象
foo = User(name="Joe",age="bar", email="joe@gmail.com")

# 输出foo中 name字段
# name字段填写的是字符串类型，但能够创建成功并输出
print(foo.age)

bar


## 2.2 使用 Pydantic 创建类

使用 Pydantic 创建类，可以对类的属性值进行格式约束。在创建类的时候会进行格式验证，如果格式不符合要求会报错。

In [5]:
# 使用 Pydantic创建pUser类别，说明age使用int类型
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [6]:
# 生成user对象
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

# 输出foo中 name字段
print(foo_p.name)

Jane


In [7]:
# 使用了pydantic，由于age不是int，因此会报错。输出报错内容
try:
    foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")
except Exception as e:
    print(e)

1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


In [8]:
class Class(BaseModel):
    students: List[pUser]

In [9]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

# 三、基于Pydantic的OpenAI函数定义

## 3.1 构造 OpenAI 的 function

我们创建了一个`WeatherSearch`类，它继承自 Pydantic 的 BaseModel 子类.因此`WeatherSearch`类的所有成员都被具备了数据类型校验机制，该类有一个str类型的成员`airport_code`表示机场代码，并有一个描述信息description，用来说明 airport_code 的作用，在 airport_code 的上方也有一段文本描述信息：输入机场代码，获取该机场的天气信息。这段文本信息是对类`WeatherSearch`的说明，意思是通过机场代码可以查询天气情况。

同时，我们使用 langchain 将这个`WeatherSearch`类转换成openai的函数描述对象：

In [10]:
# 定义WeatherSearch
# WeatherSearch的function需要写注释完成函数的description
class WeatherSearch(BaseModel):
    """输入机场代码，获取该机场的天气信息"""
    airport_code: str = Field(description="输入机场代码查询天气")

In [12]:
# 导入langchain
from langchain_core.utils.function_calling import convert_to_openai_tool

In [13]:
# 转化为openai function calling模式
weather_tool = convert_to_openai_tool(WeatherSearch)
weather_tool

{'type': 'function',
 'function': {'name': 'WeatherSearch',
  'description': '输入机场代码，获取该机场的天气信息',
  'parameters': {'properties': {'airport_code': {'description': '输入机场代码查询天气',
     'type': 'string'}},
   'required': ['airport_code'],
   'type': 'object'}}}



接下来，使用 langchain 的`convert_pydantic_to_openai_function`方法将 Pydantic 类转换成了 openai 的函数描述对象。需要的注意的是在定义Pydantic类时文本描述信息不可缺少，如缺少文本描述信息会导致转换时出错。

- `WeatherSearch1`，由于我们没有在`WeatherSearch1`中加入对本身的描述信息，导致在转换时报错。
- `WeatherSearch2`加入对本身的描述信息，因此不会报错。

In [14]:
# WeatherSearch1没有写注释，会报错
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="输入机场代码查询天气")

convert_to_openai_tool(WeatherSearch1)

{'type': 'function',
 'function': {'name': 'WeatherSearch1',
  'description': '',
  'parameters': {'properties': {'airport_code': {'description': '输入机场代码查询天气',
     'type': 'string'}},
   'required': ['airport_code'],
   'type': 'object'}}}

In [16]:
# 构造WeatherSearch2，不对参数注释
class WeatherSearch2(BaseModel):
    """输入机场代码，获取该机场的天气信息"""
    airport_code: str

convert_to_openai_tool(WeatherSearch2)

{'type': 'function',
 'function': {'name': 'WeatherSearch2',
  'description': '输入机场代码，获取该机场的天气信息',
  'parameters': {'properties': {'airport_code': {'type': 'string'}},
   'required': ['airport_code'],
   'type': 'object'}}}

## 3.2 通过langchain使用function

为了实现 LLM 对 function 的调用，有一下两种方式

- 在`invoke`中指定 functions
- 执行`invoke`之前使用`bind`方法来绑定函数描述对象

In [17]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

loaded = load_dotenv(find_dotenv(), override=True)
# 从环境变量中获取 OpenAI API Key 或者直接赋值
API_KEY = os.getenv("API_KEY")

# 如果您使用的是官方 API，就直接用 https://api.siliconflow.cn/v1 就行。
BASE_URL = "https://api.siliconflow.cn/v1"

In [18]:
model = ChatOpenAI(temperature=0, model_name="deepseek-ai/DeepSeek-V3", max_tokens=4096,
                        openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                        seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                        extra_body={
                            "enable_thinking": False
                        }
                        )

In [19]:
# 在invoke中使用openai function功能

tools = [
    weather_tool
]

model.invoke("今天旧金山的天气怎么样？", tools=tools)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0197636e501460851934f9c68067056a', 'function': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 109, 'total_tokens': 118, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197636e48f9370c793df4fffea6aa23', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f82ce302-4fcb-455a-abb8-34b93c64e931-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '0197636e501460851934f9c68067056a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 9, 'total_tokens': 118, 'input_token_details': {}, 'output_token_details': {}})

In [20]:
# 直接在构造模型中声明functions，对话时不需要在声明
model_with_function = model.bind(tools=tools)
model_with_function.invoke("今天旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0197636e8c1df54a889b7bef1f820eae', 'function': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 109, 'total_tokens': 118, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197636e866b0658a9567fef7e9503db', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2747c8df-8302-46a0-a4de-fa63bd33773c-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '0197636e8c1df54a889b7bef1f820eae', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 9, 'total_tokens': 118, 'input_token_details': {}, 'output_token_details': {}})

## 3.3 强制使用function

如果想要强制使用 function，需要在`bind`中增加`function_call`参数。

In [21]:
# 强制使用function，在模型构建时声明function_call
tool_choice={
    "type": "function",
    "function": {
     "name": "WeatherSearch"
    }
}

model_with_forced_function = model.bind(tools=tools, tool_choice=tool_choice)

In [22]:
# 基于已经声明的function对话，能够调用function
model_with_forced_function.invoke("旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0197636ec225b66af2955e3e7cd964b1', 'function': {'arguments': '{"airport_code": "SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 108, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197636ebcb49fed4e3e2aac53765d1a', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--08efe1eb-e377-44f3-a5a9-61edb26dde87-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '0197636ec225b66af2955e3e7cd964b1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 108, 'output_tokens': 9, 'total_tokens': 117, 'input_token_details': {}, 'output_token_details': {}})

In [23]:
# 输入hi，强制使用function的模型，即使prompt与函数description无关也会命中
model_with_forced_function.invoke("你好!")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0197636ed65cb7ca57b8848507e398d1', 'function': {'arguments': '{"airport_code":"PEK"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 104, 'total_tokens': 112, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197636ed1715de834c975bc64665ef2', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3c395393-bc26-44bd-b365-1e3f7d342cf6-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'PEK'}, 'id': '0197636ed65cb7ca57b8848507e398d1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 104, 'output_tokens': 8, 'total_tokens': 112, 'input_token_details': {}, 'output_token_details': {}})

In [24]:
# 输入hi，未强制使用function的模型，prompt与函数description无关不会命中
model_with_function.invoke("你好!")

AIMessage(content='你好！有什么可以帮您的吗？😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 104, 'total_tokens': 114, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197636ee11243a5c37e01090ba51e17', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--b18c9d58-885b-4945-a9ee-8a59473b81da-0', usage_metadata={'input_tokens': 104, 'output_tokens': 10, 'total_tokens': 114, 'input_token_details': {}, 'output_token_details': {}})

# 四、使用function

在一般情况下我们会使用 chain 来实现整个问答的流程，接下来我们通过创建 chain 来实现函数调用功能

## 4.1 链式使用

In [25]:
# 引入需要的环境
from langchain.prompts import ChatPromptTemplate

In [26]:
# 使用预定义模版创建聊天代码
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的助手"),
    ("user", "{input}")
])

In [27]:
# 使用prompt + model_with_function 组合
chain = prompt | model_with_function

In [28]:
# 创建命中function的问答
chain.invoke({"input": "今天旧金山的天气怎么样？"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0197636f141e4cc86daff8d9ff6848bd', 'function': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 116, 'total_tokens': 125, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197636f0db3b446e1b559180a4fb5ff', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5ec0cba3-cc90-428c-a0c9-e8f4589e410c-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '0197636f141e4cc86daff8d9ff6848bd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 116, 'output_tokens': 9, 'total_tokens': 125, 'input_token_details': {}, 'output_token_details': {}})

In [29]:
# 创建未命中function的问答
chain.invoke({"input": "你好!"})

AIMessage(content='你好！有什么可以帮您的吗？😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197636f19730d014f853ff782d312fe', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--577780c8-b347-421d-9538-47326bed33d2-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121, 'input_token_details': {}, 'output_token_details': {}})

## 4.2 使用多个function

我们可以传递一组函数，让 LLM 根据问题上下文决定使用哪个函数。

In [30]:
# 创建 ArtistSearch function
class ArtistSearch(BaseModel):
    """调用此命令可以获得特定艺术家的歌曲名称"""
    artist_name: str = Field(description="要查的艺术家的名字")
    n: int = Field(description="number of results")

In [31]:
# 组装WeatherSearch和ArtistSearch 函数
tools = [
    convert_to_openai_tool(WeatherSearch),
    convert_to_openai_tool(ArtistSearch)
]

In [32]:
# 将两个function放入模型
model_with_functions = model.bind(tools=tools)

In [33]:
# 命中 WeatherSearch function
model_with_functions.invoke("旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763700af851828996157e14e4855d', 'function': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 206, 'total_tokens': 215, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '01976370049b1e927a406a7c9127f1a1', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5a3a03c8-7fb5-4350-8787-25235f4db2a9-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': '019763700af851828996157e14e4855d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 206, 'output_tokens': 9, 'total_tokens': 215, 'input_token_details': {}, 'output_token_details': {}})

In [34]:
# 命中 ArtistSearch function
model_with_functions.invoke("找到泰勒·斯威夫特的三首歌？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0197637014c037ecc5a00dc58da326cf', 'function': {'arguments': '{"artist_name":"泰勒·斯威夫特","n":3}', 'name': 'ArtistSearch'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 210, 'total_tokens': 226, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '0197637010324411c5855e89dea8c9c6', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4d9597ba-cbd2-4c95-a4a2-d8c982f51783-0', tool_calls=[{'name': 'ArtistSearch', 'args': {'artist_name': '泰勒·斯威夫特', 'n': 3}, 'id': '0197637014c037ecc5a00dc58da326cf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 210, 'output_tokens': 16, 'total_tokens': 226, 'input_token_details': {}, 'output_token_details': {}})

In [35]:
# 命中都未命中
model_with_functions.invoke("你好!")

AIMessage(content='你好！有什么可以帮您的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 202, 'total_tokens': 210, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763701c2a48da216e655fe14ec105', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--305d7764-664f-483d-b0e8-3227eb6635b7-0', usage_metadata={'input_tokens': 202, 'output_tokens': 8, 'total_tokens': 210, 'input_token_details': {}, 'output_token_details': {}})

# 五、英文版提示

**3.1 构造OpenAI的function**

In [36]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [37]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [38]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

**4.1 链式使用**

In [39]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

**4.2 使用多个function**

In [40]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")